In [1]:
import bs4
import glob
import json
import itertools
import pandas as pd
import re
import readability
import string
import sys
import typing

from tqdm import tqdm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


topics = {}

for file in glob.glob('./data/*.json'):
    with open(file, 'r') as rfile:
        content = json.load(rfile)
        topics[content['steps'][0]['id']] = content['steps'][0]

In [2]:
def make_soup() -> None:
    soups = {}
    for _id, block in topics.items():
        text = block['block']['text']
        text = text[4:] if text.startswith('</p>') else text
        text = text[5:] if text.startswith('</ul>') else text
        soups[_id] = bs4.BeautifulSoup(text, features='lxml')
    return soups
        
soups = make_soup()

In [3]:
SPECIAL_SECTIONS = [
    re.compile(
        r'\s*<code[^>]*>[\w\W]*?</code>\s*',
        flags=re.MULTILINE,
    ),
    re.compile(
        r'\s*(<em>|<strong>)*<span class="math-tex">[\w\W]*?</span>(</em>|</strong>)*\s*',
        flags=re.MULTILINE,
    ),
]

def get_words(soup: bs4.BeautifulSoup) -> typing.List[str]:
    punct = string.punctuation.replace("'", "").replace("-", "")
    text = remove_special_sections(soup.find('body').decode_contents())
    words: typing.List[str] = []
    for word in re.sub(f'[{punct}]', ' ', text).split():
        if word[:1].islower() and len(word) > 2 and '\u200b' not in word:
            words.append(word if "'" not in word else word[:word.index("'")])

    return list(words)


def get_sentences(soup: bs4.BeautifulSoup) -> typing.List[str]:
    text = remove_special_sections(soup.find('body').decode_contents())
    return re.split('[!?.:]', text)


def remove_special_sections(content: str) -> str:
    content = content.replace('  ', '  ')
    for special in SPECIAL_SECTIONS:
        content = special.sub(' ', content)

    soup = bs4.BeautifulSoup(f'<div>{content}</div>', features='lxml')
    return soup.find('div').text


def shorten(text: str) -> str:
    if len(text) > 80:
        return text[:80] + '...'
    return text

In [4]:
def length_of_raw_text(_id: int) -> int:
    return len(soups[_id].text)

def number_of_sections(_id: int) -> int:
    return len(topics[_id]['block']['table_of_contents'])

def average_like(_id: int) -> float:
    count, value = 0, 0
    for likes in topics[_id]['likes_statistics']:
        if not likes['subject']:
            count += likes['total_count']
            value += likes['total_count'] * likes['value']
    if count == 0:
        return -10
    return value / count

def likes_count(_id: int) -> float:
    count = 0
    for likes in topics[_id]['likes_statistics']:
        if not likes['subject']:
            count += likes['total_count']
    return count

def number_of_alerts(_id: int) -> int:
    return len(soups[_id].findAll('div', {'class': 'alert'}))

def number_of_bold_elements(_id: int) -> int:
    return len(soups[_id].findAll('strong')) + len(soups[_id].findAll('b'))

def number_of_italics(_id: int) -> int:
    return len(soups[_id].findAll('em')) + len(soups[_id].findAll('i'))

def number_of_images(_id: int) -> int:
    return len(soups[_id].findAll('img'))

def number_of_snippets(_id: int) -> int:
    return len(soups[_id].select('pre code'))

def number_of_code_elements(_id: int) -> int:
    return len(soups[_id].select('code'))

def number_of_external_links(_id: int) -> int:
    return len(soups[_id].select('a'))

def average_paragraph_length(_id: int) -> int:
    ps = soups[_id].findAll('p')
    return sum(len(p.text) for p in ps) / len(ps)

def max_paragraph_length(_id: int) -> int:
    ps = soups[_id].findAll('p')
    return max(len(p.text) for p in ps)

def number_of_tables(_id: int) -> int:
    return len(soups[_id].select('table'))

def length_of_text(_id: int) -> int:
    return len(remove_special_sections(soups[_id].find('body').decode_contents()))

def average_word_length(_id: int) -> int:
    words = get_words(soups[_id])
    return sum(map(len, words)) / len(words)

def number_of_long_words(_id: int) -> int:
    words = get_words(soups[_id])
    return len(list(filter(lambda word: len(word) > 12, words)))

def dale_chall_score(_id: int) -> int:
    text = remove_special_sections(soups[_id].find('body').decode_contents())
    return readability.Readability(text).dale_chall().score
    
def flesch_score(_id: int) -> int:
    text = remove_special_sections(soups[_id].find('body').decode_contents())
    return readability.Readability(text).flesch().score

def average_sentence_length(_id: int) -> int:
    sentences = get_sentences(soups[_id])
    return sum(map(len, sentences)) / len(sentences)

def number_of_long_sentences(_id: int) -> int:
    sentences = get_sentences(soups[_id])
    return len(list(filter(lambda sentence: len(sentence) > 12, sentences)))

def average_section_length(_id: int) -> int:
    text = soups[_id].decode_contents()
    count, total_length = 0, 0
    start = end = text.find('<h2')
    while end > 0:
        end = text.find('<h2', start + 1)
        if end == -1 and (
            text.find('Conclusion', start + 1) > 0 or text.find('Summary', start + 1) > 0):
            break
        count += 1
        total_length += end - start if end != -1 else len(text) - start
        start = end
    return total_length / count if count > 0 else 1000

def max_section_length(_id: int) -> int:
    text = soups[_id].decode_contents()
    max_length = 0
    start = end = text.find('<h2')
    while end > 0:
        end = text.find('<h2', start + 1)
        length = end - start if end != -1 else len(text) - start
        max_length = max(max_length, length)
        start = end
    return max_length if max_length > 0 else 1000

def sentiment_score(_id: int) -> float:
    text = remove_special_sections(soups[_id].find('body').decode_contents())
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(text)
    return sentiment_dict['compound']

In [ ]:
prepared = {}

with tqdm(total=len(soups), file=sys.stdout) as pbar:
    for _id in soups.keys():
        pbar.update(1)
        prepared[_id] = {
            'id': _id,
            'length_of_raw_text': length_of_raw_text(_id),
            'number_of_sections': number_of_sections(_id),
            'average_like': average_like(_id),
            'number_of_alerts': number_of_alerts(_id),
            'number_of_bold_elements': number_of_bold_elements(_id),
            'number_of_italics': number_of_italics(_id),
            'number_of_images': number_of_images(_id),
            'number_of_snippets': number_of_snippets(_id),
            'number_of_code_elements': number_of_code_elements(_id),
            'number_of_links': number_of_external_links(_id),
            'average_paragraph_length': average_paragraph_length(_id),
            'max_paragraph_length': max_paragraph_length(_id),
            'number_of_tables': number_of_tables(_id),
            'length_of_text': length_of_text(_id),
            'average_word_length': average_word_length(_id),
            'number_of_long_words': number_of_long_words(_id),
            'dale_chall_score': dale_chall_score(_id),
            'flesch_score': flesch_score(_id),
            'average_sentence_length': average_sentence_length(_id),
            'number_of_long_sentences': number_of_long_sentences(_id),
            'average_section_length': average_section_length(_id),
            'max_section_length': max_section_length(_id),
            'sentiment_score': sentiment_score(_id),
        }

 91%|█████████▏| 971/1064 [01:32<00:10,  8.82it/s]

In [9]:
MARGIN = 1.2

try:
    df = pd.DataFrame(prepared.values())
except:
    with open('prepared.json') as rfile:
        df = pd.DataFrame.from_dict(json.load(rfile))

df = df[(df.average_like != -10) & (df.likes_count > 10)]

X = df.drop(
    labels=[
        'id',
        'average_like',
        'length_of_raw_text',
        'number_of_long_sentences',
        'max_section_length',
    ],
    axis=1,
)

y = df.average_like >= MARGIN

In [10]:
df[df.average_like < MARGIN].describe().iloc[:, 1:13]

,length_of_raw_text,number_of_sections,average_like,number_of_alerts,number_of_bold_elements,number_of_italics,number_of_images,number_of_snippets,number_of_code_elements,number_of_links,average_paragraph_length,max_paragraph_length
count,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
mean,6744.666667,5.027778,0.857124,0.722222,6.861111,2.750000,2.916667,4.472222,19.833333,0.277778,177.330206,562.861111
std,1965.652665,1.963759,0.324745,1.185896,7.691007,5.688962,4.016928,5.944919,23.805762,0.566246,53.951245,223.070924
min,3693.000000,0.000000,-0.096774,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,64.181818,271.000000
25%,5482.750000,4.000000,0.788194,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,144.684444,423.000000
50%,6397.500000,5.000000,0.933940,0.000000,4.000000,0.000000,1.500000,1.000000,8.500000,0.000000,178.520588,494.500000
75%,7964.250000,5.000000,1.077279,1.000000,8.250000,3.000000,3.250000,8.000000,35.250000,0.000000,206.917293,651.250000
max,11664.000000,12.000000,1.181818,4.000000,34.000000,30.000000,15.000000,21.000000,74.000000,2.000000,297.444444,1508.000000


In [11]:
df[df.average_like >= MARGIN].describe().iloc[:, 1:13]

,length_of_raw_text,number_of_sections,average_like,number_of_alerts,number_of_bold_elements,number_of_italics,number_of_images,number_of_snippets,number_of_code_elements,number_of_links,average_paragraph_length,max_paragraph_length
count,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000
mean,5058.473743,4.365363,1.742635,0.841341,7.251397,2.807821,1.366480,6.022346,23.788827,0.730726,172.709955,475.153073
std,2101.704480,1.639667,0.147162,1.168887,7.115729,5.925615,2.499171,4.811381,20.417923,1.506179,54.572664,168.020799
min,1034.000000,0.000000,1.209459,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.115385,190.000000
25%,3573.500000,3.000000,1.696529,0.000000,2.000000,0.000000,0.000000,2.000000,7.000000,0.000000,135.086081,368.000000
50%,4733.000000,4.000000,1.780488,0.000000,5.000000,0.000000,0.000000,6.000000,20.000000,0.000000,165.642857,447.000000
75%,6265.500000,5.000000,1.837838,1.000000,10.000000,3.000000,2.000000,9.000000,36.000000,1.000000,201.786199,555.500000
max,15352.000000,18.000000,2.000000,9.000000,52.000000,50.000000,16.000000,30.000000,120.000000,16.000000,514.866667,2877.000000


In [12]:
df[df.average_like < MARGIN].describe().iloc[:, 13:]

,number_of_tables,length_of_text,average_word_length,number_of_long_words,dale_chall_score,flesch_score,average_sentence_length,number_of_long_sentences,average_section_length,max_section_length,sentiment_score,likes_count
count,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
mean,0.305556,4697.805556,5.412105,5.388889,9.088161,56.788203,80.149293,55.055556,2050.420440,3301.638889,0.836503,80.416667
std,0.950772,1755.718474,0.307296,5.066855,0.761368,7.921152,10.733020,19.530115,635.692379,1098.146078,0.394893,72.549638
min,0.000000,2393.000000,4.904882,0.000000,7.795456,44.234117,59.140625,28.000000,1000.000000,1000.000000,-0.988900,11.000000
25%,0.000000,3675.500000,5.203039,2.000000,8.546546,50.314458,73.542057,44.000000,1546.375000,2687.250000,0.916850,31.750000
50%,0.000000,4128.000000,5.363191,3.000000,8.933634,57.050337,77.617788,50.500000,2064.791667,3303.000000,0.985000,52.000000
75%,0.000000,4986.750000,5.631784,8.000000,9.684564,62.916469,84.518431,58.500000,2473.312500,4034.000000,0.992725,99.000000
max,5.000000,10534.000000,6.131765,25.000000,10.747200,73.022377,109.043478,116.000000,3750.000000,6368.000000,0.999000,308.000000


In [13]:
df[df.average_like >= MARGIN].describe().iloc[:, 13:]

,number_of_tables,length_of_text,average_word_length,number_of_long_words,dale_chall_score,flesch_score,average_sentence_length,number_of_long_sentences,average_section_length,max_section_length,sentiment_score,likes_count
count,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000,895.000000
mean,0.175419,3906.910615,5.400512,4.249162,8.837966,57.844118,78.137718,46.537430,1743.404011,2551.953073,0.798569,411.959777
std,0.938524,1567.667853,0.305238,5.052901,0.749137,8.314127,12.046821,18.402062,703.506100,1122.184937,0.484833,1107.112165
min,0.000000,850.000000,4.548951,0.000000,6.581730,14.837535,29.837209,7.000000,506.500000,748.000000,-0.997700,11.000000
25%,0.000000,2744.500000,5.190424,1.000000,8.334032,52.529273,70.665306,33.000000,1265.041667,1782.000000,0.921700,59.000000
50%,0.000000,3730.000000,5.385343,3.000000,8.809471,58.434094,77.578947,45.000000,1597.750000,2358.000000,0.980500,142.000000
75%,0.000000,4862.500000,5.591288,6.000000,9.296189,63.795097,84.871711,57.000000,2088.000000,3132.500000,0.993600,391.000000
max,15.000000,13339.000000,6.375610,39.000000,11.701850,82.155818,128.764706,142.000000,6047.000000,10286.000000,0.999500,21231.000000


In [14]:
df.drop(
    labels=[
        'id',
        'length_of_raw_text',
        'number_of_long_sentences',
        'max_section_length',
    ],
    axis=1,
).corr()

,number_of_sections,average_like,number_of_alerts,number_of_bold_elements,number_of_italics,number_of_images,number_of_snippets,number_of_code_elements,number_of_links,average_paragraph_length,...,number_of_tables,length_of_text,average_word_length,number_of_long_words,dale_chall_score,flesch_score,average_sentence_length,average_section_length,sentiment_score,likes_count
number_of_sections,1.000000,-0.029367,0.198382,0.112410,0.183878,0.074647,0.347914,0.325652,0.113905,0.015561,...,0.077631,0.499703,-0.046607,0.227606,-0.039332,0.006956,-0.036228,-0.175286,0.086611,-0.055400
average_like,-0.029367,1.000000,0.016021,-0.029252,-0.053630,-0.139090,0.099526,0.108714,0.037864,-0.027726,...,-0.040976,-0.139048,-0.017820,-0.052755,-0.120760,0.041029,-0.051539,-0.143350,-0.026911,-0.005638
number_of_alerts,0.198382,0.016021,1.000000,0.088548,0.049993,-0.080718,0.171882,0.347896,0.042414,-0.171434,...,-0.014275,0.300296,0.062007,0.143824,-0.020401,-0.001200,0.014988,0.181037,0.021541,0.001110
number_of_bold_elements,0.112410,-0.029252,0.088548,1.000000,0.128207,0.063529,-0.139286,-0.132052,0.051824,0.070365,...,0.164912,0.351362,0.104187,0.168117,0.166479,-0.029721,0.015619,0.127522,0.079841,0.092506
number_of_italics,0.183878,-0.053630,0.049993,0.128207,1.000000,0.109801,-0.016436,-0.035748,0.111064,0.102513,...,0.245322,0.371028,-0.094155,0.202061,0.083442,0.012943,0.054428,0.144375,0.080212,-0.073368
number_of_images,0.074647,-0.139090,-0.080718,0.063529,0.109801,1.000000,-0.210199,-0.134675,0.067726,-0.209270,...,-0.032582,0.167626,-0.191255,0.038748,-0.046833,0.111140,-0.011666,0.002539,0.068268,-0.073140
number_of_snippets,0.347914,0.099526,0.171882,-0.139286,-0.016436,-0.210199,1.000000,0.701644,0.035601,-0.291074,...,-0.012402,0.120491,-0.114915,-0.024687,-0.188983,0.040584,-0.229884,0.163700,-0.009608,-0.037993
number_of_code_elements,0.325652,0.108714,0.347896,-0.132052,-0.035748,-0.134675,0.701644,1.000000,0.012156,-0.132434,...,-0.068347,0.254416,-0.073155,0.029220,-0.222618,0.085576,-0.148889,0.294912,0.026147,-0.084771
number_of_links,0.113905,0.037864,0.042414,0.051824,0.111064,0.067726,0.035601,0.012156,1.000000,0.060159,...,0.014809,0.243541,-0.004701,0.246947,0.064215,-0.002227,0.008034,0.058815,0.072151,0.074453
average_paragraph_length,0.015561,-0.027726,-0.171434,0.070365,0.102513,-0.209270,-0.291074,-0.132434,0.060159,1.000000,...,-0.038086,0.336836,0.017180,0.201532,-0.002741,0.040734,0.293562,0.112430,0.091713,-0.069415


In [15]:
df[df.average_like <= 1]

,id,length_of_raw_text,number_of_sections,average_like,number_of_alerts,number_of_bold_elements,number_of_italics,number_of_images,number_of_snippets,number_of_code_elements,...,average_word_length,number_of_long_words,dale_chall_score,flesch_score,average_sentence_length,number_of_long_sentences,average_section_length,max_section_length,sentiment_score,likes_count
5,6012,6253,4,0.866667,0,4,3,1,2,20,...,5.289528,2,9.681925,52.064007,97.307692,38,2173.666667,4785,0.9658,15
8,6413,6360,5,0.250000,0,3,1,7,0,0,...,5.383821,1,8.388631,68.868918,67.243243,69,2151.400000,3386,0.9357,96
61,9875,6875,5,0.777778,3,1,0,2,0,0,...,5.379775,5,9.812419,46.672419,78.933333,44,2480.250000,3718,0.9889,36
67,10251,5740,4,0.076923,0,1,0,4,0,0,...,5.346565,1,8.730937,52.696794,109.043478,44,2289.333333,2468,0.9872,13
210,9536,8333,12,0.931373,1,2,0,0,19,64,...,5.141049,3,8.933992,57.620961,73.677966,57,1004.272727,2912,0.9934,102
242,11670,8553,6,0.596639,0,4,0,15,21,62,...,5.037109,4,8.694335,56.732222,82.204082,44,2299.200000,3050,0.9981,238
446,5745,3693,5,0.995283,1,15,2,3,0,14,...,5.879397,3,10.103042,50.048570,83.372093,41,1092.200000,1587,-0.9889,212
479,4983,4637,5,0.936508,1,34,0,3,0,0,...,5.444867,5,8.532187,59.148341,87.957447,46,1424.200000,4178,0.9181,63
534,10087,5071,4,0.791667,0,0,3,0,7,33,...,6.131765,16,10.747200,44.234117,77.645833,46,1978.666667,4017,0.7298,48
540,6472,6400,4,0.729282,2,2,12,0,4,29,...,5.047697,3,8.551332,69.616830,76.317460,56,2187.000000,2305,0.3029,181


In [ ]:
df.to_json('prepared.json')